In [1]:
import os
os.chdir("../")

In [2]:
import cv2
import random

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CreatingFramesConfig:
    video_path: Path
    output_dir: Path
    frame_rate: int

In [4]:
from SignLanguageClassification.configuration import *

ModuleNotFoundError: No module named 'SignLanguageClassification'

In [3]:
def save_frames(frames, labels, output_dir, video_number):
    """
    Save frames as individual image files with corresponding labels.

    Args:
      frames: List of frames to be saved.
      labels: List of labels corresponding to the frames.
      output_dir: Directory where frames will be saved.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    
    for i, (frame, label) in enumerate(zip(frames, labels)):
        label_dir = os.path.join(output_dir, label)
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)
        cv2.imwrite(os.path.join(label_dir, f"video_{video_number}_frame_{i}.jpg"), cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    

def frames_from_video_file(video_path, n_frames, label, output_size=(1920, 1080), frame_step=15):
    """
    Creates frames from a video file and assigns a label to each frame.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      label: Label for the video file.
      output_size: Pixel size of the output frame image.
      frame_step: Step between consecutive frames.

    Return:
      A list of frames extracted from the video.
    """
    # Read each video frame by frame
    result = []
    src = cv2.VideoCapture(str(video_path))

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)
    need_length = min(1 + (n_frames - 1) * frame_step, video_length)

    start = 0
    if need_length < video_length:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    for _ in range(n_frames):
        ret, frame = src.read()
        if ret:
            frame = format_frames(frame, output_size)
            result.append(frame)
        else:
            result.append(np.zeros((output_size[1], output_size[0], 3), dtype=np.uint8))
    src.release()

    return result

def format_frames(frame, output_size):
    """
    Resize an image from a video to the specified output size.

    Args:
      frame: Image to be resized.
      output_size: Pixel size of the output frame image.

    Return:
      Resized frame.
    """
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB color space
    frame_resized = cv2.resize(frame_rgb, output_size, interpolation=cv2.INTER_LINEAR)  # Resize frame
    return frame_resized
